In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
import wave
import librosa
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import wave
import audioread
from pydub import AudioSegment
from datetime import datetime
import pickle
import librosa.display
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import ast
import re
from pydub import AudioSegment
from pydub.silence import split_on_silence
from tensorflow.keras.layers import Flatten
from tensorflow.keras import Input,Model
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.train import Checkpoint,CheckpointManager
path= 'D:\\Work\\Speaker classification\\'

In [3]:
class reading_audio_and_removing_silence():
    
    def __init__(self,path_audio_dir,path_csv_file,sample_rate=22050):
    
        self.dir_audio= path_audio_dir
        self.dir_csv_file= path_csv_file
        self.sub_path= None
        self.sample_rate= sample_rate
        
        #Reading the padnas self.dataframe
        self.dataframe= pd.read_csv(self.dir_csv_file)
        self.info= {}

        #Droping the nan values from these rows
        self.dataframe= self.dataframe[self.dataframe['Source']=='GOV']
        self.dataframe.dropna(axis=0,subset= ['Name','Link'],inplace=True)
        
        #include the path of the audio in the self.dataframe
        self.dataframe['path']= [self.dir_audio+Name+'.MP3' for Name in self.dataframe['Name'].values]


    def creating_a_dataframe(self,min_silence):   
        dataset= pd.DataFrame(columns=['Name','Gender','Features','Duration'])
        all_feature= []
        
        if self.sub_path==None:
            
            if self.dir_audio.split('\\')[-1]=='':
                self.sub_path= '\\'.join(self.dir_audio.split('\\')[:len(self.dir_audio.split('\\'))-2])

            else:
                self.sub_path= '\\'.join(self.dir_audio.split('\\')[:len(self.dir_audio.split('\\'))-1])

        #Getting the self.dataframe
        print('Creating an self.dataframe and removing the silence.....\n')

        for name in tqdm(self.dataframe['Name'].values):
            feature,duration= self.reading_a_file(name)

            #removing silence from the audio
            feature= self.remove_silence(feature,min_silence)

            self.info['Features']= feature
            self.info['Duration']= duration
            all_feature.append(feature)
            self.info['Gender']= self.dataframe[self.dataframe['Name']==name]['Sex'].values[0]
            self.info['Name']= name

            dataset= dataset.append(self.info,ignore_index=True)

        #saving the self.dataframe to the disk
        dataset.to_csv(self.sub_path+r'\features_dataset.csv')

        with open(self.sub_path+r'\features','wb') as f:
            pickle.dump(all_feature,f)

        return  
    
    
    def getduration(self,temp):
        #converting time 10:30 format to total number of seconds
        minute,second= temp.split(':')
        duration= int(minute)*60+int(second)
        return duration

    
    
    def reading_a_file(self,name):
        
        path= self.dataframe[self.dataframe['Name']==name]['path'].values[0]
        arr_audio,_=librosa.load(path,sr= self.sample_rate)
        
        if len(arr_audio.shape)>1: #dual channel audio
            #arr_audio.shape= (2,n) where 2 represent dual channel value
            arr_audio= arr_audio.sum(0)/2
            

        #Removing noises from the audio
        time_stamp= self.dataframe[self.dataframe['Name']==name]['Time Stamp'].values[0]
        
        if isinstance(time_stamp,str):

            if time_stamp=='Remove':
                #Removing first and last 1 minutes of audio
                arr= arr_audio[self.sample_rate*60:arr_audio.shape[0]-self.sample_rate*60]
                duration= arr//self.sample_rate
                
            elif len(time_stamp.split('-'))>1:
                start,end= time_stamp.split('-')

                #converting 10:40 time to seconds
                start_duration= self.getduration(start)
                end_duration= self.getduration(end)
                arr= arr_audio[self.sample_rate*start_duration:self.sample_rate*end_duration]
                duration= end_duration-start_duration

            else: 
                arr= arr_audio
                duration= arr//self.sample_rate

        return arr,duration

    
    def remove_silence(self,array_feature,min_silence):
        '''
        array_feature:array containing amplitude of an audio
        min_silence:threshould below which the silence will be removed
        '''
        
        #Step1:Create an audio_segment
        feature= self.float_to_int(array_feature)
        audio = AudioSegment(feature.tobytes(),frame_rate = self.sample_rate,sample_width = feature.dtype.itemsize\
                             ,channels = 1)
        
        #Step2:removing the silence from the audio segment
        audio_chunks = split_on_silence(audio,min_silence_len=min_silence,silence_thresh= -30,keep_silence= 100)

        #Step3:converting it back to the array
        feature= sum(audio_chunks)
        feature= np.array(feature.get_array_of_samples())
        feature= self.int_to_float(feature)
        return feature
        
        
#Citation:https://github.com/huseinzol05/malaya-speech/blob/master/malaya_speech/utils/astype.py
    def float_to_int(array, type=np.int16):

        if array.dtype == type:
            return array

        if array.dtype not in [np.int16, np.int32, np.int64]:
            if np.max(np.abs(array)) == 0:
                array[:] = 0
                array = type(array * np.iinfo(type).max)
            else:
                array = type(array / np.max(np.abs(array)) * np.iinfo(type).max)
        return array


    def int_to_float(array, type=np.float32):

        if array.dtype == type:
            return array

        if array.dtype not in [np.float16, np.float32, np.float64]:
            if np.max(np.abs(array)) == 0:
                array = array.astype(np.float32)
                array[:] = 0
            else:
                array = array.astype(np.float32) / np.max(np.abs(array))

        return array
    
    def __call__(self,min_silence):
        start= datetime.now()
        self.creating_a_dataframe(min_silence)
        print('The time requires to read an audio as array and remove the silence is:',datetime.now()-start)

In [7]:
reading_audio= reading_audio_and_removing_silence('D:\\Work\\Speaker classification\\Audio data\\AUDIO\\' ,'D:\\Work\\Speaker classification\\data.csv')
reading_audio(min_silence=400)

  0%|          | 0/34 [00:00<?, ?it/s]

Creating an self.dataframe and removing the silence.....



C:\Users\itsba\anaconda3\envs\newGPU\lib\site-packages\librosa\core\audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
  0%|          | 0/34 [00:16<?, ?it/s]


AttributeError: 'reading_audio_and_removing_silence' object has no attribute 'dtype'

In [9]:
class getting_audio_dataset():
    
    def __init__(self,path_audio_dir,path_csv_file,second,sample_rate=22050,window_size=512):
        '''
        window_size: duration at which the short time fourier transform is calculated
        for speech recognition its default value is 502 at sampling rate 22050 which 
        corresponds to 23 miliseconds
        
        path_audio_dir:path to all the audio file
        path_csv_file: path to the csv_file
        sample_rate: rate at which the audio is going to be sample
        seconds: duration during which the spectrogram is going to be computed 
        if 3 seconds than return spectrogram of 3 second of audio
        '''
        
        self.sample_rate= sample_rate
        self.window= window_size
        self.second= second
        self.dir_audio= path_audio_dir
        self.dir_csv_file= path_csv_file
        self.image_path= []
        self.image_dir= None
  
        
        #Reading the padnas self.dataframe
        self.dataframe= pd.read_csv(self.dir_csv_file)
        self.info= {}

        #Droping the nan values from these rows
        self.dataframe= self.dataframe[self.dataframe['Source']=='GOV']
        self.dataframe.dropna(axis=0,subset= ['Name','Link'],inplace=True)
        
        #include the path of the audio in the self.dataframe
        self.dataframe['path']= [self.dir_audio+Name+'.MP3' for Name in self.dataframe['Name'].values]
        

        if self.dir_audio.split('\\')[-1]=='':
            self.sub_path= '\\'.join(self.dir_audio.split('\\')[:len(self.dir_audio.split('\\'))-2])

        else:
            self.sub_path= '\\'.join(self.dir_audio.split('\\')[:len(self.dir_audio.split('\\'))-1])


#Step1: Creating a features self.dataframe by sampling the audios       
    def reading_dataframe(self):   
        '''
        reading the saved dataframe from the disk
        '''
        print('Reading the dataframe from the disk...\n')
        dataframe= pd.read_csv(self.sub_path+r'\features_dataset.csv')
        dataframe.drop(axis=1,columns='Unnamed: 0',inplace=True)

        with open(self.sub_path+r'\features','rb') as f:
            features= pickle.load(f)

        dataframe['Features']= features
        return  dataframe
    
#Step2:creating an spectrogram of the array which contains amplitude    
    def spectrogram(self,segment,path):
        #Getting the spectrogram of the audio segment

        mel_spec= librosa.feature.melspectrogram(y=segment, sr=self.sample_rate, n_mels=64,n_fft= self.window)
        
        #Converting applitude to Decibal
        mel_spec_dB = librosa.power_to_db(mel_spec, ref=np.max)
        
        #Saving the spectrogram in the disk
        fig = plt.Figure()
        canvas = FigureCanvas(fig)
        ax = fig.add_subplot()
        p = librosa.display.specshow(mel_spec_dB, ax=ax, y_axis='log', x_axis='time')
        image_path= self.sub_path+''
        fig.savefig(path)
        
        return 
    
    def save_spectrogram(self,dataframe,train=True):
        
        empty_dataframe=pd.DataFrame(columns=['id','audio1_path','audio2_path','label'])
        
        name= 'train' if train==True else 'test'
        print('Creating the spectrogram of ',name)
        if os.path.isdir(self.sub_path+r'\images')==False:
            os.mkdir(self.sub_path+r'\images')

        if self.image_dir==None:
            self.image_dir= self.sub_path+r'\images'

        if os.path.isdir(self.sub_path+'\\images\\'+name)==False:
            os.mkdir(self.sub_path+'\\images\\'+name)

        for i in tqdm(range(dataframe.shape[0])):
            audio1= dataframe['audio1'].values[i]
            audio2= dataframe['audio2'].values[i]
            audio_id= dataframe['id'].values[i]
            label= dataframe['label'].values[i]
            
            #Saving an image
            for j in range(2):
                #Getting the image path
                if j==0:
                    image_path1= self.image_dir+'\\'+name+'\\audio1_'+str(audio_id)
                    self.spectrogram(audio1,image_path1)
                else:
                    image_path2= self.image_dir+'\\'+name+'\\audio2_'+str(audio_id)
                    self.spectrogram(audio2,image_path2)
                
            empty_dataframe= empty_dataframe.append({'id':audio_id,'audio1_path':image_path1,'audio2_path':image_path2\
                                                        ,'label':label},ignore_index=True)

        return empty_dataframe
    
    def train_test_split(self):
        '''
        Splitting the audio in train(80%) and test(20%)
        '''
        
        dataframe= self.reading_dataframe()
        train_features_dataframe= pd.DataFrame(columns=['Name','Gender','Features','Duration'])
        test_features_dataframe= pd.DataFrame(columns=['Name','Gender','Features','Duration'])

        
        #Getting 20% of audio in test dataset
        
        print('Creating the train and test self.dataframe')
        for val in dataframe.values:
 
            
            train_features= val[2][:int(val[2].shape[0]*0.8)]
            train_duration= train_features.shape[0]//self.sample_rate
            
            test_features= val[2][int(val[2].shape[0]*0.8):]
            test_duration= test_features.shape[0]//self.sample_rate
            
            train_features_dataframe= train_features_dataframe.append({'Name':val[0],'Gender':val[1],'Features':train_features\
                                                            ,'Duration':train_duration},ignore_index=True)
            
            test_features_dataframe= test_features_dataframe.append({'Name':val[0],'Gender':val[1],'Features':test_features\
                                                            ,'Duration':test_duration},ignore_index=True)
       
        return train_features_dataframe,test_features_dataframe

    
    def similar_pairs(self,num_of_similar_pair,dataframe,train=True):
        '''
        Getting the similar pairs of audio segment
        '''
        if train==True:
            self.train= self.train.append(self.similar(dataframe,num_of_similar_pair,train),ignore_index=True)    
        else:
            self.test= self.test.append(self.similar(dataframe,num_of_similar_pair,train),ignore_index=True) 
        return
    
    
    def disimilar_pairs(self,num_of_disimilar_pair,dataframe,train=True):
        #Getting the disimilar pairs of audio segment
        if train==True:
            self.train= self.train.append(self.disimilar(dataframe,num_of_disimilar_pair,train),ignore_index=True)    
        else:
            self.test= self.test.append(self.disimilar(dataframe,num_of_disimilar_pair,train),ignore_index=True)  
        return

    
    def similar(self,dataframe,num_of_similar_pair,train=True):
        
        num_of_similar_pair= int(num_of_similar_pair*0.2) if train==False else num_of_similar_pair
 
        #Creating an empty self.dataframe
        empty_dataframe= pd.DataFrame(columns=['audio1','audio2','label'])
        for row in dataframe.values:

            #Creating hundred audio sample using audio of an individual speaker
            for i in range(num_of_similar_pair):    
                
                for j in range(2):
         
                    #Getting the starting index for that audio
                    start= np.random.randint(low=0,high= row[2].shape[0]-self.second*self.sample_rate)

                    #Getting an segment of audio
                    if j==0:
                        audio_segment1= (row[2][start:start+self.second*self.sample_rate])
                    else:
                        audio_segment2= (row[2][start:start+self.second*self.sample_rate])

                empty_dataframe= empty_dataframe.append({'audio1':audio_segment1\
                                               ,'audio2':audio_segment2,'label':1},ignore_index=True)

        
        return empty_dataframe
            
    
    
    def disimilar(self,dataframe,num_of_disimilar_pairs,train=True):
        
        num_of_disimilar_pairs= int(num_of_disimilar_pairs*0.2) if train==False else num_of_disimilar_pairs
        
        #Creating an empty self.dataframe
        empty_dataframe= pd.DataFrame(columns=['audio1','audio2','label'])
        
        for i in range(dataframe.shape[0]):
            other_audios_name= list(set(dataframe['Name'].values)\
                                    -set([dataframe['Name'].values[j] for j in range(dataframe['Name'].shape[0]) if j>i]))
           
            if len(other_audios_name)== dataframe.shape[0]:
                   return empty_dataframe
            
            for other_name in other_audios_name:
                    
                for j in range(num_of_disimilar_pairs): 
                    for k in range(2):   
                        
                        
                         #Getting an audio segment from one speaker
                        if k==0:
                            start= np.random.randint(low=0,high= dataframe['Features'].values[i].shape[0]-self.second*self.sample_rate)
                            audio_segment1= (dataframe['Features'].values[i][start:start+self.second*self.sample_rate])

                            
                        #Getting an audio segment from other speaker
                        else:
                            Features= dataframe[dataframe['Name']==other_name]['Features'].values[0]
      
                            start= np.random.randint(low=0,high=Features.shape[0]-self.second*self.sample_rate)
                            audio_segment2= (Features[start:start+self.second*self.sample_rate])
                    
                    #After getting the audio segment from two different spearker save it
                    empty_dataframe= empty_dataframe.append({'audio1':audio_segment1\
                                               ,'audio2':audio_segment2,'label':0},ignore_index=True)

 
    def creating_data(self,num_of_similar_pair,num_of_disimilar_pair,dataframe,use_previous=True,train=True):

        
        if train==True:
            
            #Creating the training dataset
            if os.path.isfile(self.sub_path+r'\train.csv')==False or use_previous== False:
                
                print('Creating the training dataset...')
                
                #Getting the similar pair
                self.similar_pairs(num_of_similar_pair,dataframe,train)
                
                #Getting the disimilar pair
                self.disimilar_pairs(num_of_disimilar_pair,dataframe,train)

        else:
            #Creating the testing dataset
            if os.path.isfile(self.sub_path+r'\test.csv')==False or use_previous== False:
                                   
                print('Creating the testing dataset...')
                self.similar_pairs(num_of_similar_pair,dataframe,train)
                self.disimilar_pairs(num_of_disimilar_pair,dataframe,train)         

        
        return
    
    
    def __call__(self,num_of_similar_pair,num_of_disimilar_pair,get_spectrogram=False,use_previous= True):
        
        start= datetime.now()
        self.train= pd.DataFrame(columns=['audio1','audio2','label'])
        self.test= pd.DataFrame(columns=['audio1','audio2','label'])

        '''
        num_of_similar_pair: 
        num_of_disimilar_pair:
        '''
        train_features_dataframe,test_features_dataframe= self.train_test_split()
        
        self.creating_data(num_of_similar_pair,num_of_disimilar_pair,train_features_dataframe,use_previous,train=True)      
        self.creating_data(num_of_similar_pair,num_of_disimilar_pair,test_features_dataframe,use_previous,train=False)
        
        self.train['id']= [i for i in range(self.train.shape[0])]
        self.test['id']=  [i for i in range(self.test.shape[0])]
        print('\nThe total time requires to get train test dataset is :',datetime.now()-start)
        
        if get_spectrogram==True:
            print('*******************************************************************\n')
            start= datetime.now()
            self.train= self.save_spectrogram(self.train,train=True)
            print('The spectrogram of train is saved')
            self.test= self.save_spectrogram(self.test,train=False)
            print('The spectrogram of test is saved')
            print('\nThe total time requires to save all the spectrogram is :',datetime.now()-start)
            
            self.train.to_csv(self.sub_path+r'\train.csv',index=False)
            self.test.to_csv(self.sub_path+r'\test.csv',index=False)
    
        return 
        
        def tripot

In [10]:
dataset=getting_audio_dataset(path+'Dataset\\',path+'data.csv',second=3)
dataset(num_of_similar_pair=100,num_of_disimilar_pair=100,get_spectrogram=True,use_previous=False)

Reading the dataframe from the disk...

Creating the train and test self.dataframe
Creating the training dataset...
Creating the testing dataset...

The total time requires to get train test dataset is : 0:04:56.407567
*******************************************************************

Creating the spectrogram of  train


  0%|          | 0/59500 [00:00<?, ?it/s]C:\Users\itsba\anaconda3\envs\newGPU\lib\site-packages\librosa\display.py:974: MatplotlibDeprecationWarning: The 'basey' parameter of __init__() has been renamed 'base' since Matplotlib 3.3; support for the old name will be dropped two minor releases later.
  scaler(mode, **kwargs)
C:\Users\itsba\anaconda3\envs\newGPU\lib\site-packages\librosa\display.py:974: MatplotlibDeprecationWarning: The 'linthreshy' parameter of __init__() has been renamed 'linthresh' since Matplotlib 3.3; support for the old name will be dropped two minor releases later.
  scaler(mode, **kwargs)
C:\Users\itsba\anaconda3\envs\newGPU\lib\site-packages\librosa\display.py:974: MatplotlibDeprecationWarning: The 'linscaley' parameter of __init__() has been renamed 'linscale' since Matplotlib 3.3; support for the old name will be dropped two minor releases later.
  scaler(mode, **kwargs)
  0%|          | 1/11900 [00:00<25:19,  7.83it/s]

The spectrogram of train is saved
Creating the spectrogram of  test


100%|██████████| 11900/11900 [24:55<00:00,  7.96it/s]


The spectrogram of test is saved

The total time requires to save all the spectrogram is : 6:28:57.485044


In [3]:
class creating_batch:
    
    def __init__(self,path_to_train_csv,path_to_test_csv,height,width,batch):
        '''Argument:
        batch:batch size
        path_to_train_csv:path to the train csv file which contains path of pair of image with their correspoing labels
        '''
        
        self.batch= batch
        self.path_to_train_csv= path_to_train_csv
        self.path_to_test_csv= path_to_test_csv
        self.height= height
        self.width= width
        
    def read(self,path,label):
        # 1)reading the content of the path
        content1= tf.io.read_file(path[0])
        content2= tf.io.read_file(path[1])
        
        # 2)decoding the content
        image1= tf.io.decode_png(content1,channels=3)
        image1= tf.image.convert_image_dtype(image1, tf.float32)
        

        image2= tf.io.decode_png(content2,channels=3)
        image2= tf.image.convert_image_dtype(image2, tf.float32)
        
        #Reshazing the image
        image1= tf.image.resize(image1,[self.height,self.width])
        image2= tf.image.resize(image2,[self.height,self.width])
        

        
        #3) Normalizing the image
#         image1= preprocessing.Normalization()(image1)
#         image2= preprocessing.Normalization()(image2)
        
        return image1,image2,label

    
    def pipeline(self,paths,label):

        dataset=tf.data.Dataset.from_tensor_slices((paths,label))
       #reading the images.....................................................
        dataset= dataset.map(self.read,num_parallel_calls=tf.data.experimental.AUTOTUNE)

       #Creating the batch
        dataset= dataset.batch(batch_size=self.batch,drop_remainder=True)

       #All the batches will be stored in the cache after the first iteration
        dataset.cache()

       #shuffeling 
        dataset=dataset.shuffle(buffer_size=batch)
        dataset = dataset.repeat()

        dataset.prefetch(tf.data.experimental.AUTOTUNE)   

        return dataset

    
    
    def create_tensorflow_dataset(self):
        
        '''
        Create a tensorflow dataset for training and test
        '''

        #Reading the train test dataframe

        train= pd.read_csv(self.path_to_train_csv)
        
        #shuffle the train and test dataframe
        train= train.sample(frac=1).reset_index(drop=True)
        
        a= np.array([train['audio1_path'].values,train['audio2_path'].values])
        b =np.moveaxis(a,0,-1)
    
        #Creating the train dataset
        train_dataset= self.pipeline(b,train['label'].values) 
        
        test= pd.read_csv(self.path_to_test_csv)
        test= test.sample(frac=1).reset_index(drop=True)
        
        a= np.array([test['audio1_path'].values,test['audio2_path'].values])
        b =np.moveaxis(a,0,-1)
    
        #Creating the train dataset
        test_dataset= self.pipeline(b,test['label'].values) 
        
        
        return train_dataset,test_dataset

        
        